# Viagem do zezinho

Este trabalho tem como objetivo descrever alguns problemas que o Zezinho teve em sua viagem e a partir dos algoritmos estudados podemos ajudá-lo.

Os problemas da viagem são os seguintes:

- Achar o menor caminho para o seu destino, utilizando o algoritmo de Dijkstra;
- Guardar seus brinquedos no cofre do hotel, onde ele tem uma capacidade especifica, seus brinquedos tem tamanhos específicos e o valor sentimental de cada um deles (Knapsack - PD);
- Verificar quais são as atrações mais próximas (Par de pontos mais próximos - Greedy). 